Install pdfminer.six or pdfminer3 (https://github.com/gwk/pdfminer3/) install: pip install pdfminer3 I switched to pdfminer3 when I upgraded to 3.7 from 3.6 I use on ubuntu and macos with python 3.7.3

pdfminer3 comes with two handy tools: pdf2txt.py and dumppdf.py examine the source. Fairly small and easy to understand.

Following is a working example (once the location of the pdf file is added)

In [2]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import re

In [3]:
url = "http://www.iacis.org/iis/author_index.php"
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")
print(soup.prettify()[:4000])

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <!-- page title -->
  <title>
   International Association for Computer Information Systems
  </title>
  <link href="../css/core.css" rel="stylesheet" type="text/css"/>
  <meta content="International Association for Computer Information Systems is a nonprofit association dedicated to the improvement of information systems and the education of information systems and computer professionals. These goals are accomplished through various activities, recognition awards, and publications." name="description"/>
  <meta content="Information systems, computer professionals, information systems education, information technology, awards, publications, IS journals, IS conference, academic journal, international conference, journal of computer information systems, iss

In [5]:
my_div = soup.find("div",{"class":"column column-650 column-left"})
print(my_div.prettify()[:2000]) 


<div class="column column-650 column-left">
 <h3>
  IIS - Author Index Search Results
 </h3>
 <hr color="#2A4580" noshade=""/>
 <br/>
 <h4>
  Results for Author:
 </h4>
 <a href="/iis/2019/1_iis_2019_1-11.pdf">
  /iis/2019/1_iis_2019_1-11.pdf
 </a>
 <br/>
 <a href="/iis/2019/1_iis_2019_12-21.pdf">
  /iis/2019/1_iis_2019_12-21.pdf
 </a>
 <br/>
 <a href="/iis/2019/1_iis_2019_22-31.pdf">
  /iis/2019/1_iis_2019_22-31.pdf
 </a>
 <br/>
 <a href="/iis/2019/1_iis_2019_32-39.pdf">
  /iis/2019/1_iis_2019_32-39.pdf
 </a>
 <br/>
 <a href="/iis/2019/1_iis_2019_40-48.pdf">
  /iis/2019/1_iis_2019_40-48.pdf
 </a>
 <br/>
 <a href="/iis/2019/1_iis_2019_49-59.pdf">
  /iis/2019/1_iis_2019_49-59.pdf
 </a>
 <br/>
 <a href="/iis/2019/1_iis_2019_60-69.pdf">
  /iis/2019/1_iis_2019_60-69.pdf
 </a>
 <br/>
 <a href="/iis/2019/1_iis_2019_70-79.pdf">
  /iis/2019/1_iis_2019_70-79.pdf
 </a>
 <br/>
 <a href="/iis/2019/1_iis_2019_80-87.pdf">
  /iis/2019/1_iis_2019_80-87.pdf
 </a>
 <br/>
 <a href="/iis/2019/1_iis_2019_8

In [6]:
my_links = soup.findAll('a')
my_links[40]

<a href="/iis/2019/2_iis_2019_83-88.pdf">/iis/2019/2_iis_2019_83-88.pdf</a>

In [35]:
links = [link.get("href") for link in my_links]
links[0:20]

['http://www.iacis.org/iis/2019/4_iis_2019_188-197.pdf',
 'http://www.iacis.org/iis/2015/4_iis_2015_183-193.pdf',
 'http://iacis.org/iis/2014/94_iis_2014_98-102.pdf',
 'http://iacis.org/iis/2014/93_iis_2014_89-97.pdf',
 'http://iacis.org/iis/2014/62_iis_2014_8-13.pdf',
 'http://iacis.org/iis/2014/92_iis_2014_77-88.pdf',
 'http://iacis.org/iis/2014/91_iis_2014_67-76.pdf',
 'http://iacis.org/iis/2014/88_iis_2014_59-66.pdf',
 'http://iacis.org/iis/2014/86_iis_2014_49-58.pdf',
 'http://iacis.org/iis/2014/29_iis_2014_450-463.pdf',
 'http://iacis.org/iis/2014/113_iis_2014_441-449.pdf',
 'http://iacis.org/iis/2014/18_iis_2014_431-440.pdf',
 'http://iacis.org/iis/2014/10_iis_2014_421-430.pdf',
 'http://iacis.org/iis/2014/149_iis_2014_411-420.pdf',
 'http://iacis.org/iis/2014/148_iis_2014_399-410.pdf',
 'http://iacis.org/iis/2014/146_iis_2014_393-398.pdf',
 'http://iacis.org/iis/2014/85_iis_2014_39-48.pdf',
 'http://iacis.org/iis/2014/144_iis_2014_383-392.pdf',
 'http://iacis.org/iis/2014/141_i

In [8]:


from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io

resource_manager = PDFResourceManager()
fake_file_handle = io.StringIO()
converter = TextConverter(resource_manager, fake_file_handle)
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('file-1.pdf', 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
        page_interpreter.process_page(page)
        break

    text = fake_file_handle.getvalue()


# close open handles
converter.close()
fake_file_handle.close()

#print(text)

Issues in Information Systems Volume 20, Issue 1, pp. 178-185, 2019    178 PriVacy: THE MISSING “V” IN BIG DATA ANALYTICS Christopher B. Davison, Ball State University, cbdavison@bsu.edu Allen D. Truell, Ball State University, atruell@bsu.edu Edward J. Lazaros, Ball State University, ejlazaros@bsu.edu  Jensen J. Zhao, Ball State University, jzhao@bsu.edu     ABSTRACT  The descriptive models of Big Data found in the research literature generally neglect privacy as a component of big data.   Utilizing Cavoukian’s  (2011) Privacy by Design framework as a foundation, the authors propose an additional “V” to the Big Data descriptive model: priVacy.  The purpose of this paper is to present an expanded theoretical model of Big Data with privacy as a necessary component.  The implication is that privacy is an aspect of many data science domains including data analytics and cybersecurity with many concomitant legal and political ramifications.  Keywords: Privacy, Big Data, Cybersecurity, Data M

In [14]:
text.find("Keywords: ")


950

In [17]:
text2 = text[text.find("Keywords: "):]
text2

'Keywords: Privacy, Big Data, Cybersecurity, Data Model, Design, Data Analytics.     INTRODUCTION  Big data is a ubiquitous facet of modern life. Big data and data analytics provide competitive advantage and profitability to organizations.  As such, the market-driven rush to obtain this data often results in overlooked privacy considerations.  Consider the IBM Watson Health analytics that aggregates massive amounts of lifestyle and behavior data related to healthcare. This data is evaluated for socioeconomic factors and provided to insurance companies (Allen, 2018).  In this paper, the authors draw upon current research on big data models and privacy research to create a privacy cognizant big data model.    To present privacy in the big data model context, the authors begin this paper by providing a literature review.  The literature review will set the context and provide definitions associated with big data and information privacy. Following that, an expanded big data framework will 

In [18]:
text3 =text2[:text2.find(".")]
text3

'Keywords: Privacy, Big Data, Cybersecurity, Data Model, Design, Data Analytics'

In [19]:
text4 = text3[10:]
text4

'Privacy, Big Data, Cybersecurity, Data Model, Design, Data Analytics'